In [0]:
!wget -c https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip
!unzip MURA-v1.1.zip
!rm MURA-v1.1.zip

In [0]:
!pip uninstall keras
!pip3 install git+https://github.com/keras-team/keras
!pip uninstall keras-preprocessing
!pip3 install git+https://github.com/keras-team/keras-preprocessing

In [0]:
!ls
#RESTART RUNTIME BEFORE PROCEEDING

In [0]:
import pandas as pd
import numpy as np
import os, time, signal, shutil
import multiprocessing as mp
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import keras
from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.densenet import DenseNet169
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.nasnet import NASNetMobile
from keras.preprocessing import image
from keras.applications.mobilenetv2 import preprocess_input
from keras.applications import MobileNet
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard)
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.metrics import binary_accuracy, binary_crossentropy
from keras.models import Model, Sequential
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenetv2 import MobileNetV2
from keras.preprocessing import image as k_im_prep
from keras import backend as K
from keras_preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [0]:
# from os import listdir
# from os.path import isfile, join
# onlyfiles = [f for f in listdir("MURA-v1.1")]
# print(onlyfiles)

## SETUP

In [0]:
# just run the cell to be able to upload and download from Dropbox
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
!chmod +x Dropbox-Uploader/dropbox_uploader.sh
source = 'Dropbox-Uploader'
dest1 = '/content'
shutil.move(source+'/'+'dropbox_uploader.sh', dest1)
!echo "pzFqTa1uWVAAAAAAAAAACtGcVohdBzoFRrwtLYsQCxhD9a7IjevvuOTNV8OWB2LX" > token.txt

In [0]:
!bash dropbox_uploader.sh download README.md

In [0]:
# download train and valid csv files that include images path and their labels
!bash dropbox_uploader.sh download train_paths_labels.csv
!bash dropbox_uploader.sh download valid_paths_labels.csv
# the saved model file
model_file= 'model.h5'

In [0]:
# watch_ monitors the state of certain file if it is modified and uploads it to Dropbox if so
def watch_(file, interval):
  from datetime import datetime
  first_Time=False
  while True:
    if os.path.isfile(file):
      if not first_Time:
        os.system("bash dropbox_uploader.sh upload "+file+" "+file)
        first_Time=True
      moddate = os.stat(file)[8]
      time.sleep(interval)
      moddate_ = os.stat(file)[8]
      if moddate < moddate_:
        os.system("bash dropbox_uploader.sh upload "+file+" "+file)
    else:
      time.sleep(interval)
      

## MODEL

In [0]:
def make_FT_model(base=1, imagenet=True, freeze_all=True, add_denses=True):
  
  #weights of pretrained model
  if (imagenet==True):
    w='imagenet'
  else:
    w=None
  
  #default because refrenced before assignment error, just scroll down
  base_model = MobileNetV2(input_shape= (224, 224, 3),weights=w, include_top=False)
  
  #initializing pretrained model
  if (base==0):
    base_model = MobileNetV2(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 1):
    base_model = DenseNet169(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 2):
    base_model = InceptionV3(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 3):
    base_model = ResNet50(input_shape= (224, 224, 3),weights=w, include_top=False)   
  elif (base == 4):
    base_model = NASNetMobile(input_shape= (224, 224, 3),weights=w, include_top=False)
    
 
  if (freeze_all):
    #freeze layers of densenet
    for layer in base_model.layers:
      layer.trainable= False 
  
  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  
  if(add_denses):
    # let's add a fully-connected layer
    #x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    #x = Dense(32, activation='relu')(x)
    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(1, activation='sigmoid')(x)
    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
  else:
    # just feature extractor
    model = Model(inputs=base_model.input, output=x)
  
  
  return model

bug: if called fn on another base model it still loads the model

In [0]:
from keras.models import load_model
def evaluate_limps(model=1,epoch=5,batch=32, imagenet=True, freeze_all=False,verbose=2):
  
  df_train=pd.read_csv('train_paths_labels.csv')
  df_valid=pd.read_csv('valid_paths_labels.csv')
  
  datagen=ImageDataGenerator(rescale=1./255)
  train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=None,x_col="Img_Path", y_col="Label", class_mode="binary", target_size=(32,32), batch_size=batch)
  valid_generator=datagen.flow_from_dataframe(dataframe=df_valid, directory=None,x_col="Img_Path", y_col="Label", class_mode="binary", target_size=(32,32), batch_size=batch)
  print(len(train_generator))
  print("making model")
  if not os.path.isfile(model_file):
    model=make_FT_model(base= model, imagenet=imagenet, freeze_all=freeze_all, add_denses=True)
  else:
    model= load_model(model_file)
  
  print("compiling")
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  
  checkpoint= ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=False,
                              save_weights_only=False, mode='auto', period=1)
  callbacks_list = [checkpoint]
    
  step_train=train_generator.n//train_generator.batch_size
  step_valid=valid_generator.n//valid_generator.batch_size
  model.fit_generator(generator=train_generator, steps_per_epoch=step_train, epochs=epoch, validation_data=valid_generator, validation_steps=step_valid, shuffle=True, verbose=verbose, callbacks=callbacks_list)
   #
  
  
  loss_tr, accuracy_tr =model.evaluate_generator(train_generator, use_multiprocessing=True,steps=step_train)
  print("training loss/accuracy: ", loss_tr,'/', accuracy_tr)

  loss_val, accuracy_val = model.evaluate_generator(valid_generator, use_multiprocessing=True,steps=step_valid)
  print("validation loss/accuracy: ", loss_val,'/', accuracy_val)
    
    

In [0]:
# #KAREEM'S COPY OF FUNCTION JUST COMMENT THE CELL AND EVERYTHING IS NORMAL
# def evaluate_limps(model=1,epoch=5,batch=32, imagenet=True, freeze_all=False,verbose=2):
  
#   df_train=pd.read_csv('train_paths_labels.csv')
#   df_valid=pd.read_csv('valid_paths_labels.csv')
  
#   datagen = ImageDataGenerator(  rescale=1./255,
#     featurewise_center=True,  #CHANGED IT TO TRUE # set input mean to 0 over the dataset
#     samplewise_center=False,  # set each sample mean to 0
#     featurewise_std_normalization=True,  #CHANGED IT TO TRUE# divide inputs by std of the dataset
#     samplewise_std_normalization=False,  # divide each input by its std
#     zca_whitening=False,  # apply ZCA whitening
#     rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
#     width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#     height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#     horizontal_flip=True,  # randomly flip images
#     vertical_flip=False,
#     zoom_range=0.1,
#     channel_shift_range=0.,
#     fill_mode='nearest')
  
#   datagen.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
#   datagen.std  = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
#   train_generator=datagen.flow_from_dataframe(dataframe=df_train, directory=None,x_col="Img_Path",
#                                               y_col="Label", class_mode="binary", target_size=(224,224), batch_size=batch)
#   valid_generator=datagen.flow_from_dataframe(dataframe=df_valid, directory=None,x_col="Img_Path",
#                                               y_col="Label", class_mode="binary", target_size=(224,224), batch_size=batch)
#   print(len(train_generator))
#   print("making model")
#   if not os.path.isfile(model_file):
#     model=make_FT_model(base= model, imagenet=imagenet, freeze_all=freeze_all, add_denses=True)
#   else:
#     model= load_model(model_file)
  
#   print("compiling")
#   model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  
#   checkpoint= ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
#   callbacks_list = [checkpoint]
    
#   step_train=train_generator.n//train_generator.batch_size
#   step_valid=valid_generator.n//valid_generator.batch_size
#   model.fit_generator(generator=train_generator, steps_per_epoch=step_train, epochs=epoch,
#                       validation_data=valid_generator, validation_steps=step_valid, shuffle=True,
#                       verbose=verbose, callbacks=callbacks_list)
#    #
  
  
#   loss_tr, accuracy_tr =model.evaluate_generator(train_generator, use_multiprocessing=True,steps=step_train)
#   print("training loss/accuracy: ", loss_tr,'/', accuracy_tr)

#   loss_val, accuracy_val = model.evaluate_generator(valid_generator, use_multiprocessing=True,steps=step_valid)
#   print("validation loss/accuracy: ", loss_val,'/', accuracy_val)
    
    

In [0]:
!rm model.h5

In [0]:
!ls

In [0]:
p = mp.Process(target=watch_, args=(model_file,5))
p.start()
evaluate_limps(model=0,epoch=1,batch=32,imagenet=True,freeze_all=False,verbose=1)


Found 36808 images belonging to 2 classes.
Found 3197 images belonging to 2 classes.
1151
making model
51879936/51877672 [==============================] - 2s 0us/step
compiling
Epoch 1/1
1150/1150 [==============================] - 1488s 1s/step - loss: 0.6316 - acc: 0.6550 - val_loss: 0.9110 - val_acc: 0.6146

Epoch 00001: saving model to model.h5


In [0]:
p = mp.Process(target=watch_, args=(model_file,5))
p.start()
#BUG : CHOSE ANOTHER MODEL , YET LOADED THE OLD ONE
#NOT BUG: i told it so, it always loads the last saved version regradless of the model
evaluate_limps(model=0,epoch=3,batch=64,imagenet=True,freeze_all=False,verbose=1)


In [0]:
!ls

In [0]:
from google.colab import files


files.download('model.h5')

In [0]:
from keras.models import load_model
p = mp.Process(target=watch_, args=(model_file,5))
p.start()
#BUG : CHOSE ANOTHER MODEL , YET LOADED THE OLD ONE
evaluate_limps(model=0,epoch=3,batch=64,imagenet=True,freeze_all=False,verbose=1)
